In [1]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier
from imblearn.over_sampling import SMOTE
import pandas as pd
from sklearn.metrics import precision_recall_curve,auc, roc_auc_score

In [2]:
train_df = pd.read_csv('label_df_train.csv')

In [3]:
train_df

,transcript_id,transcript_position,5-mers,mean_dwell_time_-1,mean_dwell_time_0,mean_dwell_time_1,weighted_mean_-1,weighted_mean_0,weighted_mean_1,weighted_sd_-1,...,5-mer_window1_GACTA,5-mer_window1_GACTC,5-mer_window1_GACTG,5-mer_window1_GACTT,A_freq,C_freq,G_freq,T_freq,gene_id,label
0,ENST00000000233,244,AAGACCA,0.008264,0.009373,0.007345,123.762870,125.793483,80.775369,5.209304,...,0.0,0.0,0.0,0.0,4,2,1,0,ENSG00000004059,0
1,ENST00000000233,261,CAAACTG,0.006609,0.006813,0.007710,109.924484,108.101783,94.108586,3.910096,...,0.0,0.0,0.0,0.0,3,2,1,1,ENSG00000004059,0
2,ENST00000000233,316,GAAACAG,0.007570,0.007416,0.007555,105.450998,99.426169,89.309704,3.400684,...,0.0,0.0,0.0,0.0,4,1,2,0,ENSG00000004059,0
3,ENST00000000233,332,AGAACAT,0.010620,0.008632,0.006101,129.548782,97.842815,89.096953,6.542432,...,0.0,0.0,0.0,0.0,4,1,1,1,ENSG00000004059,0
4,ENST00000000233,368,AGGACAA,0.010701,0.011479,0.010019,118.217577,121.925694,84.996204,7.490865,...,0.0,0.0,0.0,0.0,4,1,2,0,ENSG00000004059,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97035,ENST00000641834,610,AGAACTG,0.008452,0.005721,0.006148,128.770870,100.748672,92.027406,9.036699,...,0.0,0.0,0.0,0.0,3,1,2,1,ENSG00000167747,1
97036,ENST00000641834,697,GTGACCT,0.006091,0.007621,0.005713,97.163336,121.322246,79.123033,4.906109,...,0.0,0.0,0.0,0.0,1,2,2,2,ENSG00000167747,0
97037,ENST00000641834,864,TGGACAG,0.008525,0.009855,0.006462,114.186314,111.571903,74.535120,3.981475,...,0.0,0.0,0.0,0.0,2,1,3,1,ENSG00000167747,1
97038,ENST00000641834,926,AGGACAT,0.011754,0.012544,0.008119,115.767571,115.702294,76.807752,6.176761,...,0.0,0.0,0.0,0.0,3,1,2,1,ENSG00000167747,1


In [4]:
train_df = train_df.drop(columns = ['transcript_id', 'transcript_position','gene_id','5-mers'])

In [6]:
X_train, y_train = train_df[[i for i in train_df.columns if i!='label']],train_df['label']

In [7]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)

In [8]:
oversample = SMOTE(sampling_strategy=0.33)
X_train_scaled_res,y_train_res = oversample.fit_resample(X_train_scaled,y_train)

In [9]:
y_train_res.value_counts()

0    92643
1    30572
Name: label, dtype: int64

In [ ]:
mlp_clf = MLPClassifier(hidden_layer_sizes=(4,4), tol=1e-4, verbose = 2)
mlp_clf.fit(X_train_scaled_res,y_train_res)

In [16]:
df_valid = pd.read_csv('label_df_valid.csv')

In [19]:
X_test = df_valid[[i for i in X_train.columns]]

In [ ]:
X_test

In [21]:
X_test_scaled = scaler.transform(X_test)

In [22]:
label_probabilities = mlp_clf.predict_proba(X_test_scaled)

In [23]:
label_probabilities

array([[9.94145761e-01, 5.85423855e-03],
       [9.99969197e-01, 3.08029889e-05],
       [4.79728980e-01, 5.20271020e-01],
       ...,
       [9.85734195e-01, 1.42658052e-02],
       [9.63035985e-01, 3.69640153e-02],
       [4.97064382e-01, 5.02935618e-01]])

In [24]:
df_valid['pred_proba'] = label_probabilities[:,1]

In [25]:
precision, recall, thresholds = precision_recall_curve(df_valid['label'],df_valid['pred_proba'])
auc(recall,precision)

0.41864483869075214

In [30]:
roc_auc_score(df_valid['label'],df_valid['pred_proba'])

0.8797994028977527